In [ ]:
#Cell 0: Install the community detection package (only needs to be performed once)
!pip install python-louvain

# Cell 1: Import dependencies and read data to build a NetworkX directed graph
import pandas as pd
import networkx as nx
import community as community_louvain  # It should now import successfully.

# Modify to your Excel path
input_path = r'C:\Users\51195\Desktop\MR链式表编号后.xlsx'

# Read two columns A→B and remove duplicates
df = pd.read_excel(input_path, usecols=['A','B'], engine='openpyxl').drop_duplicates()
edges = list(df.itertuples(index=False, name=None))

# Construct a directed graph (nodes are automatically unique, no additional merging is required)
G = nx.DiGraph()
G.add_edges_from(edges)

# Calculating node degrees and community groups
degrees   = dict(G.degree())
partition = community_louvain.best_partition(G.to_undirected())

# Cell 2: Map string nodes to integer indices, prepare Bokeh GraphRenderer
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine, LabelSet, ColumnDataSource
from bokeh.plotting import figure, output_file, show

# Index Mapping
nodes     = list(G.nodes())
index_map = {node: i for i, node in enumerate(nodes)}

# GraphRenderer Example
graph = GraphRenderer()

# Node Data Source
node_source = ColumnDataSource(data=dict(
    index     = [index_map[n] for n in nodes],
    label     = nodes,
    degree    = [degrees[n] for n in nodes],
    community = [partition[n] for n in nodes],
))
graph.node_renderer.data_source = node_source
graph.node_renderer.glyph = Circle(size='degree', fill_color='community')

# Edge Data Source
start_indices = [index_map[u] for u, v in G.edges()]
end_indices   = [index_map[v] for u, v in G.edges()]
graph.edge_renderer.data_source.data = dict(start=start_indices, end=end_indices)
graph.edge_renderer.glyph = MultiLine(line_alpha=0.6, line_width=1)

# Cell 3: Layout, labeling, and outputting HTML
from bokeh.io import output_notebook
from bokeh.models import StaticLayoutProvider

# If you preview in Notebook
output_notebook()

# Computational layout
pos = nx.spring_layout(G, k=0.3, iterations=30, seed=42)
layout = { index_map[node]: coord for node, coord in pos.items() }
graph.layout_provider = StaticLayoutProvider(graph_layout=layout)

# Create Canvas (English title)
plot = figure(title="Interactive Chain Relationship Network", 
              x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="pan,wheel_zoom,box_zoom,reset", active_scroll='wheel_zoom',
              width=800, height=800)

# Adding Networks and Nodes
plot.renderers.append(graph)

# Adding Text Labels
label_source = ColumnDataSource(data=dict(
    x     = [layout[index_map[n]][0] for n in nodes],
    y     = [layout[index_map[n]][1] for n in nodes],
    label = nodes
))
labels = LabelSet(x='x', y='y', text='label', source=label_source,
                  text_font_size="8pt", text_align='center')
plot.add_layout(labels)

# Output to HTML
output_file(r'C:\Users\51195\Desktop\network.html')
show(plot)
